_**Using Genius.com API + Beautiful Soup to get song lyrics and write them to a local text file.**_

Citation: https://medium.com/analytics-vidhya/how-to-scrape-song-lyrics-a-gentle-python-tutorial-5b1d4ab351d2

Getting my Genius.com API key: https://docs.genius.com/#/getting-started-h1

In [7]:
GENIUS_API_TOKEN='5la2M_pYH7rZ653TL8ulhRnTwi6Gyy7RfhKtK5wp0tcG3xilwiWfuhTSHni5keuP'

In [8]:
# Make HTTP requests
import requests

# Scrape data from an HTML document
from bs4 import BeautifulSoup

# I/O
import os

# Search and manipulate strings
import re

1. Get a list of Genius.com URL’s for a specified number of songs for an artist

In [9]:
# Get artist object from Genius API
def request_artist_info(artist_name, page):
    base_url = 'https://api.genius.com'
    headers = {'Authorization': 'Bearer ' + GENIUS_API_TOKEN}
    search_url = base_url + '/search?per_page=10&page=' + str(page)
    data = {'q': artist_name}
    response = requests.get(search_url, data=data, headers=headers)
    return response

# Get Genius.com song url's from artist object
def request_song_url(artist_name, song_cap):
    page = 1
    songs = []
    
    while True:
        response = request_artist_info(artist_name, page)
        json = response.json()
        # Collect up to song_cap song objects from artist
        song_info = []
        for hit in json['response']['hits']:
            if artist_name.lower() in hit['result']['primary_artist']['name'].lower():
                song_info.append(hit)
    
        # Collect song URL's from song objects
        for song in song_info:
            if (len(songs) < song_cap):
                url = song['result']['url']
                songs.append(url)
            
        if (len(songs) == song_cap):
            break
        else:
            page += 1
        
    print('Found {} songs by {}'.format(len(songs), artist_name))
    return songs
    
# DEMO
# request_song_url('Lana Del Rey', 2)

In [10]:
request_song_url('Bob Dylan', 2)

Found 2 songs by Bob Dylan


['https://genius.com/Bob-dylan-murder-most-foul-lyrics',
 'https://genius.com/Bob-dylan-blowin-in-the-wind-lyrics']

2. Fetch lyrics from the URL’s


In [12]:
# Scrape lyrics from a Genius.com song URL
def scrape_song_lyrics(url):
    page = requests.get(url)
    html = BeautifulSoup(page.text, 'html.parser')
    lyrics = html.find('div', class_='lyrics').get_text()
    #remove identifiers like chorus, verse, etc
    lyrics = re.sub(r'[\(\[].*?[\)\]]', '', lyrics)
    #remove empty lines
    lyrics = os.linesep.join([s for s in lyrics.splitlines() if s])         
    return lyrics

# DEMO
# print(scrape_song_lyrics('https://genius.com/Lana-del-rey-young-and-beautiful-lyrics'))

In [13]:
scrape_song_lyrics('https://genius.com/Bob-dylan-murder-most-foul-lyrics')

'\'Twas a dark day in Dallas, November \'63\nA day that will live on in infamy\nPresident\u2005Kennedy\u2005was a-ridin\' high\nGood\u2005day to be livin\' and a\u2005good day to die\nBeing led to the slaughter like a sacrificial lamb\nHe said, "Wait a minute, boys, you know who I am?"\n"Of course we do, we know who you are"\nThen they blew off his head while he was still in the car\nShot down like a dog in broad daylight\nWas a matter of timing and the timing was right\nYou got unpaid debts, we\'ve come to collect\nWe\'re gonna kill you with hatred, without any respect\nWe\'ll mock you and shock you and we\'ll grin in your face\nWe\'ve already got someone here to take your place\nThe day they blew out the brains of the king\nThousands were watching, no one saw a thing\nIt happened so quickly, so quick, by surprise\nRight there in front of everyone\'s eyes\nGreatest magic trick ever under the sun\nPerfectly executed, skillfully done\nWolfman, oh Wolfman, oh Wolfman, howl\nRub-a-dub-dub

In [14]:
# trying print view instead of returning as above

print (scrape_song_lyrics('https://genius.com/Bob-dylan-murder-most-foul-lyrics'))

'Twas a dark day in Dallas, November '63
A day that will live on in infamy
President Kennedy was a-ridin' high
Good day to be livin' and a good day to die
Being led to the slaughter like a sacrificial lamb
He said, "Wait a minute, boys, you know who I am?"
"Of course we do, we know who you are"
Then they blew off his head while he was still in the car
Shot down like a dog in broad daylight
Was a matter of timing and the timing was right
You got unpaid debts, we've come to collect
We're gonna kill you with hatred, without any respect
We'll mock you and shock you and we'll grin in your face
We've already got someone here to take your place
The day they blew out the brains of the king
Thousands were watching, no one saw a thing
It happened so quickly, so quick, by surprise
Right there in front of everyone's eyes
Greatest magic trick ever under the sun
Perfectly executed, skillfully done
Wolfman, oh Wolfman, oh Wolfman, howl
Rub-a-dub-dub, it's a murder most foul
Hush, little children, you

3. Loop through all URL’s and write lyrics to one file


In [22]:
def write_lyrics_to_file(artist_name, song_count):
    f = open('lyrics/' + artist_name.lower() + '.txt', 'wb')
    urls = request_song_url(artist_name, song_count)
    for url in urls:
        lyrics = scrape_song_lyrics(url)
        f.write(lyrics.encode("utf8"))
    f.close()
    num_lines = sum(1 for line in open('lyrics/' + artist_name.lower() + '.txt', 'rb'))
    print('Wrote {} lines to file from {} songs'.format(num_lines, song_count))
  
# DEMO  
# write_lyrics_to_file('Kendrick Lamar', 100)

In [24]:
write_lyrics_to_file('Bob Dylan', 500)

Found 500 songs by Bob Dylan
Wrote 18015 lines to file from 500 songs
